In [4]:
from google.colab import files
uploaded = files.upload()


Saving SignImage48x48.zip to SignImage48x48.zip


In [6]:
import zipfile
import os

with zipfile.ZipFile("SignImage48x48.zip", 'r') as zip_ref:
    zip_ref.extractall("SignImage48x48")


In [14]:
# ✅ Step 1: Fix the folder structure if nested
import shutil
import os

source = "/content/SignImage48x48/SignImage48x48"
target = "/content/SignImage48x48"

if os.path.exists(source):
    for folder in os.listdir(source):
        shutil.move(os.path.join(source, folder), target)
    shutil.rmtree(source)

print("✅ Folder structure fixed.")

# ✅ Step 2: Import Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle

# ✅ Step 3: Load Data
data_dir = '/content/SignImage48x48'
categories = sorted([folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))])

X = []
y = []

img_size = 48

print("📦 Loading data...")
for idx, category in enumerate(categories):
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            resized_array = cv2.resize(img_array, (img_size, img_size))
            X.append(resized_array)
            y.append(idx)
        except Exception as e:
            pass  # Optionally print(e)

X = np.array(X).reshape(-1, img_size, img_size, 1) / 255.0
y = to_categorical(np.array(y), num_classes=len(categories))

print(f"✅ Data loaded: {len(X)} images across {len(categories)} categories.")

# ✅ Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Step 5: Build Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# ✅ Step 6: Train Model
print("🏋️ Training model...")
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# ✅ Step 7: Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🎯 Test accuracy: {test_acc * 100:.2f}%")

# ✅ Step 8: Save Model as model.p using Pickle
print("💾 Saving model as model.p ...")
with open('model.p', 'wb') as f:
    pickle.dump(model, f)

print("✅ model.p saved successfully.")


✅ Folder structure fixed.
📦 Loading data...
✅ Data loaded: 40 images across 27 categories.
🏋️ Training model...
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0312 - loss: 3.2747 - val_accuracy: 0.0000e+00 - val_loss: 3.2753
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.0625 - loss: 3.2308 - val_accuracy: 0.0000e+00 - val_loss: 3.3088
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1250 - loss: 3.1893 - val_accuracy: 0.2500 - val_loss: 3.3275
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1250 - loss: 3.1147 - val_accuracy: 0.0000e+00 - val_loss: 3.3684
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.0625 - loss: 3.2471 - val_accuracy: 0.0000e+00 - val_loss: 3.3718
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.0938 - loss: 3.1661 - val_accuracy: 0.0000e+00 - val_loss: 3.3631
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.0938 - loss: 3.0908 - val_accuracy: 0.0000e+00 - val_l